In [1]:
import numpy as np
import matplotlib.pylab as plt
from scipy.io import loadmat, whosmat
import datetime
import pandas as pd
from pandas import DataFrame

In [2]:
battery = 'B0005'
data_set = loadmat("../data/mat/"+battery+".mat")
print(type(data_set))

<class 'dict'>


In [3]:
for i in data_set:
    print(i)

__header__
__version__
__globals__
B0005


In [4]:
chdataset = []
disdatset = []
impedataset = []
capacity_data = []
rectidataset = []
reNrct_data = []
cnt = 0 # 정렬을 위함
for cycle_len in range(len(data_set[battery][0, 0]['cycle'][0])):
    row = data_set[battery][0, 0]['cycle'][0, cycle_len] # 각 사이클에 대한 데이터 불러오기
    if row['type'][0] == 'charge':        
        
        # 전체 날짜
        date_time =  datetime.datetime(int(row['time'][0][0]),
                               int(row['time'][0][1]),
                               int(row['time'][0][2]),
                               int(row['time'][0][3]),
                               int(row['time'][0][4])) 
        + datetime.timedelta(seconds=int(row['time'][0][5]))
        
        # 주변 온도
        ambient_temperature = row['ambient_temperature'][0][0]
        
        # 각 데이터
        data = row['data']
        for index in range(len(data[0][0]['Voltage_measured'][0])):
            voltage_measured = data[0][0]['Voltage_measured'][0][index]
            current_measured = data[0][0]['Current_measured'][0][index]
            temperature_measured = data[0][0]['Temperature_measured'][0][index]
            current_charge = data[0][0]['Current_charge'][0][index]
            voltage_charge = data[0][0]['Voltage_charge'][0][index]
            time = data[0][0]['Time'][0][index]
            
            chdataset.append([cnt,
                              'charge',
                              cycle_len,
                              date_time,
                              ambient_temperature,
                              voltage_measured,
                              current_measured,
                              temperature_measured,
                              current_charge,
                              voltage_charge,
                              time])
            cnt=cnt+1
            
    elif row['type'][0] == 'discharge':
        # 전체 날짜
        date_time =  datetime.datetime(int(row['time'][0][0]),
                               int(row['time'][0][1]),
                               int(row['time'][0][2]),
                               int(row['time'][0][3]),
                               int(row['time'][0][4])) 
        + datetime.timedelta(seconds=int(row['time'][0][5]))
        
        # 주변 온도
        ambient_temperature = row['ambient_temperature'][0][0]
        
        # 각 데이터
        data = row['data']
        
        # 배터리 용량
        capacity = data[0][0]['Capacity'][0][0]
        
        for index in range(len(data[0][0]['Voltage_measured'][0])):
            voltage_measured = data[0][0]['Voltage_measured'][0][index]
            current_measured = data[0][0]['Current_measured'][0][index]
            temperature_measured = data[0][0]['Temperature_measured'][0][index]
            current_load = data[0][0]['Current_load'][0][index]
            voltage_load = data[0][0]['Voltage_load'][0][index]
            time = data[0][0]['Time'][0][index]
            
            disdatset.append([cnt,
                              'discharge',
                              cycle_len,
                              date_time,
                              ambient_temperature,
                              voltage_measured,
                              current_measured,
                              temperature_measured,
                              current_load,
                              voltage_load,
                              time])
            capacity_data.append([cnt,
                                  'discharge',
                                  cycle_len,
                                  date_time,
                                  ambient_temperature,
                                  capacity])
            cnt=cnt+1
    else:
        data = row['data']
        
        re = data[0][0]['Re'][0][0]
        rct = data[0][0]['Rct'][0][0]
        
        re_im_len = len(data[0][0]['Rectified_Impedance'])
        sen_c_len = len(data[0][0]['Sense_current'][0])
        
        if re_im_len != sen_c_len:
            for index in range(len(data[0][0]['Rectified_Impedance'])):
                rectified_impedance = data[0][0]['Rectified_Impedance'][index][0]
                re_im_real = np.real(rectified_impedance)
                re_im_imag = np.imag(rectified_impedance)
                
                rectidataset.append([cnt,
                                     'impedance',
                                     cycle_len,
                                     re_im_real,
                                     re_im_imag])
                cnt=cnt+1
            
        for index in range(len(data[0][0]['Sense_current'][0])):
            sense_current = data[0][0]['Sense_current'][0][index]
            sense_real = np.real(sense_current)
            sense_imag = np.imag(sense_current)
                
            battery_current = data[0][0]['Battery_current'][0][index]
            battery_c_real = np.real(battery_current)
            battery_c_imag = np.imag(battery_current)
                
            current_ratio = data[0][0]['Current_ratio'][0][index]
            ratio_real = np.real(current_ratio)
            ratio_imag = np.imag(current_ratio)
                
            battery_impedance = data[0][0]['Battery_impedance'][index][0]
            battery_im_real = np.real(battery_impedance)
            battery_im_imag = np.imag(battery_impedance)
                
            impedataset.append([cnt,
                                'impedance',
                                cycle_len,
                                sense_real,
                                sense_imag,
                                battery_c_real,
                                battery_c_imag,
                                ratio_real,
                                ratio_imag,
                                battery_im_real,
                                battery_im_imag])
                
            reNrct_data.append([cnt,
                                'impedance',
                                cycle_len,
                                re,
                                rct])
            cnt=cnt+1

In [5]:
chdf = pd.DataFrame(data=chdataset,
                    columns =['cnt',
                              'type',
                              'cycle',
                              'date_time',
                              'ambient_temperature',
                              'voltage_measured',
                              'current_measured',
                              'temperature_measured',
                              'current_charge',
                              'voltage_charge',
                              'time'])
disdf = pd.DataFrame(data=disdatset,
                    columns =['cnt',
                              'type',
                              'cycle',
                              'date_time',
                              'ambient_temperature',
                              'voltage_measured',
                              'current_measured',
                              'temperature_measured',
                              'current_charge',
                              'voltage_charge',
                              'time'])
capacity = pd.DataFrame(data=capacity_data,
                       columns = ['cnt',
                                  'type',
                                  'cycle',
                                  'data_time',
                                  'ambient_temperature',
                                  'capacity'])

imdf = pd.DataFrame(data=impedataset,
                       columns = ['cnt',
                                  'type',
                                  'cycle',
                                  'sense_real',
                                  'snese_imag',
                                  'battery_c_real',
                                  'battery_c_imag',
                                  'ratio_real',
                                  'ratio_imag',
                                  'battery_im_real',
                                  'battery_im_imag'])

reimdf = pd.DataFrame(data=rectidataset,
                       columns = ['cnt',
                                  'type',
                                  'cycle',
                                  're_im_real',
                                  're_im_imag'])

rercdf = pd.DataFrame(data=reNrct_data,
                       columns = ['cnt',
                                  'type',
                                  'cycle',
                                  're',
                                  'rct'])

In [6]:
chdf

,cnt,type,cycle,date_time,ambient_temperature,voltage_measured,current_measured,temperature_measured,current_charge,voltage_charge,time
0,0,charge,0,2008-04-02 13:08:00,24,3.873017,-0.001201,24.655358,0.000,0.003,0.000
1,1,charge,0,2008-04-02 13:08:00,24,3.479394,-4.030268,24.666480,-4.036,1.570,2.532
2,2,charge,0,2008-04-02 13:08:00,24,4.000588,1.512731,24.675394,1.500,4.726,5.500
3,3,charge,0,2008-04-02 13:08:00,24,4.012395,1.509063,24.693865,1.500,4.742,8.344
4,4,charge,0,2008-04-02 13:08:00,24,4.019708,1.511318,24.705069,1.500,4.753,11.125
...,...,...,...,...,...,...,...,...,...,...,...
541168,615639,charge,615,2008-05-28 11:09:00,24,0.236356,-0.003484,23.372048,0.000,0.003,0.000
541169,615640,charge,615,2008-05-28 11:09:00,24,0.003365,-0.001496,23.369434,0.000,0.003,2.547
541170,615641,charge,615,2008-05-28 11:09:00,24,4.985137,0.000506,23.386535,0.000,5.002,5.500
541171,615642,charge,615,2008-05-28 11:09:00,24,4.984720,0.000442,23.386983,-0.002,5.002,8.312


In [7]:
disdf

,cnt,type,cycle,date_time,ambient_temperature,voltage_measured,current_measured,temperature_measured,current_charge,voltage_charge,time
0,789,discharge,1,2008-04-02 15:25:00,24,4.191492,-0.004902,24.330034,-0.0006,0.000,0.000
1,790,discharge,1,2008-04-02 15:25:00,24,4.190749,-0.001478,24.325993,-0.0006,4.206,16.781
2,791,discharge,1,2008-04-02 15:25:00,24,3.974871,-2.012528,24.389085,-1.9982,3.062,35.703
3,792,discharge,1,2008-04-02 15:25:00,24,3.951717,-2.013979,24.544752,-1.9982,3.030,53.781
4,793,discharge,1,2008-04-02 15:25:00,24,3.934352,-2.011144,24.731385,-1.9982,3.011,71.922
...,...,...,...,...,...,...,...,...,...,...,...
50280,615547,discharge,613,2008-05-27 20:45:00,24,3.579262,-0.001569,34.864823,0.0006,0.000,2781.312
50281,615548,discharge,613,2008-05-27 20:45:00,24,3.581964,-0.003067,34.814770,0.0006,0.000,2791.062
50282,615549,discharge,613,2008-05-27 20:45:00,24,3.584484,-0.003079,34.676258,0.0006,0.000,2800.828
50283,615550,discharge,613,2008-05-27 20:45:00,24,3.587336,0.001219,34.565580,0.0006,0.000,2810.640


In [8]:
capacity

,cnt,type,cycle,data_time,ambient_temperature,capacity
0,789,discharge,1,2008-04-02 15:25:00,24,1.856487
1,790,discharge,1,2008-04-02 15:25:00,24,1.856487
2,791,discharge,1,2008-04-02 15:25:00,24,1.856487
3,792,discharge,1,2008-04-02 15:25:00,24,1.856487
4,793,discharge,1,2008-04-02 15:25:00,24,1.856487
...,...,...,...,...,...,...
50280,615547,discharge,613,2008-05-27 20:45:00,24,1.325079
50281,615548,discharge,613,2008-05-27 20:45:00,24,1.325079
50282,615549,discharge,613,2008-05-27 20:45:00,24,1.325079
50283,615550,discharge,613,2008-05-27 20:45:00,24,1.325079


In [9]:
imdf

,cnt,type,cycle,sense_real,snese_imag,battery_c_real,battery_c_imag,ratio_real,ratio_imag,battery_im_real,battery_im_imag
0,22939,impedance,40,-1.000000,1.000000,-1.000000,1.000000,1.000000,0.000000,-0.438926,-0.107298
1,22940,impedance,40,820.609497,-36.234550,337.091461,-82.920776,2.320415,0.463305,0.130088,-0.197115
2,22941,impedance,40,827.242188,-48.231228,330.631561,-70.013718,2.424193,0.367465,0.058771,0.033307
3,22942,impedance,40,827.193481,-56.195717,330.808624,-61.734425,2.447002,0.286778,0.005814,-0.060547
4,22943,impedance,40,824.929504,-53.241478,332.682678,-57.629013,2.434305,0.261646,0.126081,-0.090444
...,...,...,...,...,...,...,...,...,...,...,...
13339,615634,impedance,614,915.489014,-64.045120,230.149506,91.909889,3.334835,-1.610038,0.245024,-0.049836
13340,615635,impedance,614,916.725525,2.986217,212.188858,107.745811,3.440393,-1.732898,0.264594,-0.055235
13341,615636,impedance,614,914.619629,126.111481,176.598038,131.682785,3.670656,-2.022960,0.288571,-0.058837
13342,615637,impedance,614,880.340820,293.825287,136.847626,146.881027,4.060164,-2.210749,0.317700,-0.059127


In [10]:
reimdf

,cnt,type,cycle,re_im_real,re_im_imag
0,22900,impedance,40,0.070069,-4.799847e-04
1,22901,impedance,40,0.068179,-1.190041e-03
2,22902,impedance,40,0.067933,-5.682681e-05
3,22903,impedance,40,0.066918,-8.787264e-04
4,22904,impedance,40,0.068071,-1.974802e-04
...,...,...,...,...,...
10837,615586,impedance,614,0.051287,-9.816256e-04
10838,615587,impedance,614,0.048027,-3.750248e-04
10839,615588,impedance,614,0.047851,-9.521025e-04
10840,615589,impedance,614,0.049161,8.241281e-07


In [11]:
rercdf

,cnt,type,cycle,re,rct
0,22939,impedance,40,0.044669,0.069456
1,22940,impedance,40,0.044669,0.069456
2,22941,impedance,40,0.044669,0.069456
3,22942,impedance,40,0.044669,0.069456
4,22943,impedance,40,0.044669,0.069456
...,...,...,...,...,...
13339,615634,impedance,614,0.050036,0.074792
13340,615635,impedance,614,0.050036,0.074792
13341,615636,impedance,614,0.050036,0.074792
13342,615637,impedance,614,0.050036,0.074792


In [12]:
cycling_data = pd.concat([chdf, disdf])
cycling_data.sort_values(by='cnt', inplace=True) # 원본에도 영향 미치게 정렬
cycling_data.pop('cnt') #정렬 후 삭제
cycling_data

,type,cycle,date_time,ambient_temperature,voltage_measured,current_measured,temperature_measured,current_charge,voltage_charge,time
0,charge,0,2008-04-02 13:08:00,24,3.873017,-0.001201,24.655358,0.000,0.003,0.000
1,charge,0,2008-04-02 13:08:00,24,3.479394,-4.030268,24.666480,-4.036,1.570,2.532
2,charge,0,2008-04-02 13:08:00,24,4.000588,1.512731,24.675394,1.500,4.726,5.500
3,charge,0,2008-04-02 13:08:00,24,4.012395,1.509063,24.693865,1.500,4.742,8.344
4,charge,0,2008-04-02 13:08:00,24,4.019708,1.511318,24.705069,1.500,4.753,11.125
...,...,...,...,...,...,...,...,...,...,...
541168,charge,615,2008-05-28 11:09:00,24,0.236356,-0.003484,23.372048,0.000,0.003,0.000
541169,charge,615,2008-05-28 11:09:00,24,0.003365,-0.001496,23.369434,0.000,0.003,2.547
541170,charge,615,2008-05-28 11:09:00,24,4.985137,0.000506,23.386535,0.000,5.002,5.500
541171,charge,615,2008-05-28 11:09:00,24,4.984720,0.000442,23.386983,-0.002,5.002,8.312


In [13]:
cycling_data.to_csv(battery+".csv", index=None)